In [1]:
import numpy as np

In [8]:
# data = np.load("./sim_data/A1/A1_SBM_K4_dense_splits_seed42.npz")
data = np.load("./sim_data/B1/B1_TranslationInvariant_RBF_ARD_splits_seed42.npz")
data.keys()

KeysView(NpzFile './sim_data/B1/B1_TranslationInvariant_RBF_ARD_splits_seed42.npz' with keys: train, val, test)

In [21]:
data["test"].shape, data["val"].shape, data["train"].shape

((6600, 2), (6600, 2), (52802, 2))

In [17]:
# nodes = np.load("./sim_data/A1/A1_SBM_K4_dense_nodes.npz")
# nodes["positions"], nodes["blocks"]
nodes = np.load("./sim_data/B1/B1_TranslationInvariant_RBF_ARD_nodes.npz")
nodes["positions"].shape

(10000, 4)

In [19]:
# nodes["positions"].shape, nodes["blocks"].shape

In [20]:
nodes

NpzFile './sim_data/B1/B1_TranslationInvariant_RBF_ARD_nodes.npz' with keys: positions